# **Philippine Scam SMS**
**Phase 2: Natural Language Processing**

**Author/s: [Anton Reyes](https://www.github.com/AGR-yes)**

## **Introduction**

### **Requirements and Imports**

#### **Imports**

**Basic Libraries**

* `numpy` contains a large collection of mathematical functions
* `pandas` contains functions that are designed for data manipulation and data analysis

In [ ]:
import numpy as np
import pandas as pd

**Visualization Libraries**

* `matplotlib.pyplot` contains functions to create interactive plots
* `seaborn` is a library based on matplotlib that allows for data visualization
* `plotly` is an open-source graphing library for Python.

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

**Natural Language Processing Libraries**
* `re` is a module that allows the use of regular expressions

In [ ]:
import re

#### **Datasets and Files**

The following `csv` file was used for this project:

- `.csv` contains (insert short description)

## **Data Collection**

Importing the dataset using pandas.

In [ ]:
dataset = ".csv"

df = pd.read_csv(dataset)
df.head()

## **Description of the Dataset**

Here, we find the shape of the dataset.

In [ ]:
df.shape

By looking at the `info` of the dataframe, we can see that there are `non-null` values. 

In [ ]:
df.info()

## **Exploratory Data Analysis**

## **Data Preprocessing**

#### **Data Preprocessing**

#### **Data Cleaning**

#### **Feature Selection**

# **Saving Dataframes as CSVs**

In [ ]:
#.to_csv('.csv')
